# RQ1: How many higher education institutions are found in counties with majority underrepresented groups?
Answer: 659

## What are the characteristics of those institutions?
- More private-for-profit
- More 2-year (vs 4-year or above)

In [ ]:
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt

from tools import tree
from pathlib import Path
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

today

In [ ]:
RAW_DATA = Path("../data/raw/")
INTERIM_DATA = Path("../data/interim/")
PROCESSED_DATA = Path("../data/processed/")
FINAL_DATA = Path("../data/final/")
EXTERNAL_DATA = Path("../data/external/")

In [ ]:
tree(PROCESSED_DATA)

In [ ]:
counties_data = pd.read_csv(PROCESSED_DATA / 'counties.csv')
institutions_data = pd.read_csv(PROCESSED_DATA / 'institutions_data.csv')
counties_shapes = gpd.read_file(PROCESSED_DATA / 'geodata' / 'tl_2019_us_county.shp')

In [ ]:
counties_data.head().T

In [ ]:
institutions_data.head().T

In [ ]:
counties_shapes.head().T

In [ ]:
counties_shapes.sample(5)

In [ ]:
counties_shapes.plot();

In [ ]:
import us

In [ ]:
contiguous_fips = [state.fips for state in us.STATES_CONTIGUOUS]

In [ ]:
mask_contiguous_fips = counties_shapes['STATEFP'].isin(contiguous_fips)

In [ ]:
mask_contiguous_fips

In [ ]:
counties_shapes = counties_shapes[mask_contiguous_fips]

In [ ]:
name_to_fips_map = us.states.mapping("name", "fips")

In [ ]:
institutions_data['fips_state_code'] = institutions_data['fips_state_code'].map(name_to_fips_map)

In [ ]:
mask_contiguous_fips_institutions = institutions_data['fips_state_code'].isin(contiguous_fips)

In [ ]:
institutions_data = institutions_data[mask_contiguous_fips_institutions]

In [ ]:
counties_data.head()

In [ ]:
counties_data['share_underrepresented'] = (
    counties_data['black_alone'] 
    + counties_data['latino_alone'] 
    + counties_data['american_indian_and_alaska_native'] 
    + counties_data['native_hawaiian_and_pacific_islander']
) / counties_data['universe']

In [ ]:
counties_data.head().T

In [ ]:
subset_counties_data = counties_data[['geoid', 'name', 'share_underrepresented']].copy()
subset_counties_shapes = counties_shapes[['GEOID', 'NAME', 'geometry']].copy()

In [ ]:
subset_counties_data['geoid'] = subset_counties_data['geoid'].astype(str).str.zfill(5)

In [ ]:
subset_counties_data = subset_counties_data.set_index('geoid')
subset_counties_shapes = subset_counties_shapes.set_index('GEOID')

In [ ]:
working_gdf = subset_counties_shapes.join(subset_counties_data)

In [ ]:
working_gdf.plot(column = 'share_underrepresented');

In [ ]:
gplt.choropleth(working_gdf, projection=gcrs.WebMercator(), hue = 'share_underrepresented'); 

In [ ]:
geo_institutions = gpd.GeoDataFrame(institutions_data, geometry = gpd.points_from_xy(institutions_data['longitude'], institutions_data['latitude']))

In [ ]:
working_gdf.crs

In [ ]:
geo_institutions.crs = working_gdf.crs

In [ ]:
geo_institutions.crs

In [ ]:
gplt.pointplot(geo_institutions)

In [ ]:
ax = gplt.choropleth(working_gdf, projection=gcrs.WebMercator(), hue = 'share_underrepresented', figsize = (12,12))
gplt.pointplot(geo_institutions, ax = ax, zorder = 3, alpha = 0.3, color = "red", s = 2)

In [ ]:
mask_majority_underrepresented = working_gdf['share_underrepresented'] > 0.50

In [ ]:
working_gdf[mask_majority_underrepresented].plot();

In [ ]:
working_gdf[~mask_majority_underrepresented].plot();

In [ ]:
majority_underrepresented = working_gdf[mask_majority_underrepresented].copy()

In [ ]:
majority_underrepresented.head()

In [ ]:
institutions_in_majority_underrepresented = gpd.sjoin(geo_institutions, majority_underrepresented, how="inner", op="intersects")

In [ ]:
ax = gplt.choropleth(working_gdf, projection=gcrs.WebMercator(), hue = 'share_underrepresented', figsize = (12,12))
gplt.pointplot(institutions_in_majority_underrepresented, ax = ax, zorder = 3, alpha = 0.3, color = "red", s = 2)

In [ ]:
majority_underrepresented.shape

In [ ]:
counties_shapes.shape

In [ ]:
geo_institutions.shape

In [ ]:
institutions_in_majority_underrepresented.shape

In [ ]:
institutions_in_majority_underrepresented.head().T

In [ ]:
institutions_in_majority_underrepresented['control'].value_counts(normalize = True)

In [ ]:
geo_institutions['control'].value_counts(normalize = True)

In counties where there's a majority of underrepresented groups, 29% of higher ed institutions are private for-profits whereas across the united states that number drops to 19%. 

In [ ]:
institutions_in_majority_underrepresented['level'].value_counts(normalize = True)

In [ ]:
geo_institutions['level'].value_counts(normalize = True)

# Checkpoint

In [ ]:
tree(PROCESSED_DATA)

In [ ]:
PROCESSED_DATA.joinpath("processed_geodata").mkdir()

In [ ]:
PROCESSED_DATA.joinpath("processed_institutions").mkdir()

In [ ]:
tree(PROCESSED_DATA)

In [ ]:
working_gdf.to_file(PROCESSED_DATA / 'processed_geodata' / 'contiguous_us.shp')
geo_institutions.to_file(PROCESSED_DATA / 'processed_institutions' / 'geo_institutions.shp')